# Data wrangling, grouping and aggregation

Next, we will continue working with weather data, but expand our analysis to cover longer periods of data from Finland. In the following, you will learn various useful techniques in pandas to manipulate, group and aggregate the data in different ways that are useful when extracting insights from your data. In the end, you will learn how to create an automated data analysis workflow that can be repeated with multiple input files having a similar structure. As a case study, we will investigate whether January 2020 was the warmest month on record also in Finland, as the month was the warmest one on record globally [^noaanews]. 

## Cleaning data while reading

In this section we are using weather observation data from Finland that was downloaded from NOAA (see `Datasets` chapter for further details). The input data is separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

By looking at the data, we can notice a few things that we need to consider when reading the data:

1. **Delimiter:** The columns are separated with a varying amount of spaces which requires using some special tricks when reading the data with pandas `read_csv()` function
2. **NoData values:** NaN values in the NOAA data are coded with varying number of `*` characters, hence, we need to be able to instruct pandas to interpret those as NaNs. 
3. **Many columns**: The input data contains altogether 33 columns. Many of those do not contain any meaningful data for our needs. Hence, we should probably ignore the unnecessary columns already at this stage. 

Handling and cleaning heterogeneous input data (such as our example here) could naturally be done after the data has been imported to a DataFrame. However, in many cases, it is actually useful to do some cleaning and preprocessing already when reading the data. In fact, that is often much easier to do. In our case, we can read the data with varying number of spaces between the columns (1) by using a parameter `delim_whitespace=True` (alternatively, specifying `sep='\s+'` would work). For handling the NoData values (2), we can tell pandas to consider the `*` characters as NaNs by using a paramater `na_values` and specifying a list of characters that should be converted to NaNs. Hence, in this case we can specify `na_values=['*', '**', '***', '****', '*****', '******']` which will then convert the varying number of `*` characters into NaN values. Finally, we can limit the number of columns that we read (3) by using the `usecols` parameter, which we already used previously. In our case, we are interested in columns that might be somehow useful to our analysis (or at least meaningful to us), including e.g. the station name, timestamp, and data about the wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

Achieving all these things is pretty straightforward using the `read_csv()` function: 

In [1]:
import pandas as pd

# Define relative path to the file
fp = 'data/029820.txt'

# Read data using varying amount of spaces as separator, 
# specifying '*' characters as NoData values, 
# and selecting only specific columns from the data
data = pd.read_csv(fp, delim_whitespace=True, 
                   na_values=['*', '**', '***', '****', '*****', '******'],
                   usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN']
                  )

Let's see now how the data looks by printing the first five rows with the `head()` function:

In [2]:
data.head()

,USAF,YR--MODAHRMN,DIR,SPD,GUS,TEMP,MAX,MIN
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN


Perfect, looks good. We have skipped a bunch of unnecessary columns and also the asterisk (\*) characters have been correctly converted to NaN values.  

### Renaming columns

Let's take a closer look at the column names of our DataFrame. A description for all these columns is available in the metadata file [data/3505doc.txt](data/3505doc.txt). 

In [3]:
data.columns

Index(['USAF', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'TEMP', 'MAX', 'MIN'], dtype='object')

As we see, some of the column names are a bit awkward and difficult to interpret. Luckily, it is easy to alter labels in a pandas DataFrame using the `rename()` function. In order to change the column names, we need to tell pandas how we want to rename the columns using a dictionary that converts the old names to new ones. As you probably remember from Chapter 1, a `dictionary` is a specific data structure in Python for storing key-value pairs.

We can define the new column names using a dictionary where we list "`key: value`" pairs in following manner:
   
- `USAF`: `STATION_NUMBER`
- `YR--MODAHRMN`: `TIME`
- `SPD`: `SPEED`
- `GUS`: `GUST`
- `TEMP`: `TEMP_F`

Hence, the original column name (e.g. `YR--MODAHRMN`) is the dictionary `key` which will be converted to a new column name `TIME` (which is the `value`). The temperature values in our data file is again represented in Fahrenheit. We will soon convert these temperatures to Celsius. Hence, in order to avoid confusion with the columns, let's rename the column `TEMP` to `TEMP_F`. Also the station number `USAF` is much more intuitive if we call it `STATION_NUMBER`.

In [4]:
# Create the dictionary with old and new names
new_names = {'USAF':'STATION_NUMBER', 'YR--MODAHRMN': 'TIME', 
             'SPD': 'SPEED', 'GUS': 'GUST', 
             'TEMP': 'TEMP_F'
            }
new_names

{'USAF': 'STATION_NUMBER',
 'YR--MODAHRMN': 'TIME',
 'SPD': 'SPEED',
 'GUS': 'GUST',
 'TEMP': 'TEMP_F'}

Our dictionary looks correct, so now we can change the column names by passing that dictionary using the parameter `columns` in the `rename()` function:

In [5]:
# Rename the columns
data = data.rename(columns=new_names)
data.columns

Index(['STATION_NUMBER', 'TIME', 'DIR', 'SPEED', 'GUST', 'TEMP_F', 'MAX',
       'MIN'],
      dtype='object')

Perfect, now our column names are easier to understand and use. 

## Apply: How to use functions with pandas

Now it's time to convert those temperatures from Fahrenheit to Celsius. We have done this many times before, but this time we will learn how to apply our own functions to data in a pandas DataFrame. We will define a function for the temperature conversion, and apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called `TEMP_C`.

But first, it is a good idea to check some basic properties of our new input data before proceeding with data analysis.

In [6]:
# First rows
data.head(2)

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN


In [7]:
# Last rows
data.tail(2)

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
198332,29820,201910012200,287.0,21.0,25.0,47.0,NaN,NaN
198333,29820,201910012300,287.0,33.0,41.0,46.0,NaN,NaN


In [8]:
# Data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198334 entries, 0 to 198333
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   STATION_NUMBER  198334 non-null  int64  
 1   TIME            198334 non-null  int64  
 2   DIR             193660 non-null  float64
 3   SPEED           196436 non-null  float64
 4   GUST            26649 non-null   float64
 5   TEMP_F          197916 non-null  float64
 6   MAX             29868 non-null   float64
 7   MIN             29536 non-null   float64
dtypes: float64(6), int64(2)
memory usage: 12.1 MB


Nothing suspicous for the first and last rows, but here with `info()` we can see that the number of observations per column seem to be varying if you compare the `Non-Null Count` information to the number of entries in the data (N=198334). Only station number and time seem to have data on each row. All other columns seem to have some missing values. This is not necessarily anything dangerous, but good to keep in mind. 

In [9]:
# Descriptive stats
data.describe()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
count,198334.0,1.983340e+05,193660.000000,196436.000000,26649.000000,197916.000000,29868.000000,29536.000000
mean,29820.0,1.990974e+11,199.798033,14.794462,23.895606,43.717845,46.405852,40.537446
std,0.0,2.691914e+09,95.299190,8.482308,9.231637,14.306138,14.446547,14.350235
min,29820.0,1.906010e+11,1.000000,0.000000,11.000000,-28.000000,-20.000000,-28.000000
25%,29820.0,1.979011e+11,120.000000,9.000000,17.000000,34.000000,36.000000,31.000000
50%,29820.0,1.997061e+11,212.000000,14.000000,22.000000,43.000000,45.000000,40.000000
75%,29820.0,2.013113e+11,270.000000,20.000000,29.000000,55.000000,59.000000,52.000000
max,29820.0,2.019100e+11,990.000000,97.000000,73.000000,90.000000,102.000000,77.000000


By looking at the `TEMP_F` values (Fahrenheit temperatures), we can confirm that our measurements seems more or less valid because the value range of the temperatures makes sense, i.e. there are no outliers such as extremely high `MAX` values or low `MIN` values. It is always a good practice to critically check your data before doing any analysis, as it is possible that your data may include incorrect values, e.g. due to a sensor malfunction or human error. 

### Defining function for pandas

Now we are sure that our data looks okay, and we can start our temperature conversion process by first defining our temperature conversion function from Fahrenheit to Celsius. Pandas can use regular functions, hence you can define functions for pandas exactly in the same way as you would do normally (as we learned in Chapter 1). Hence, let's define a function that converts Fahrenheits to Celsius: 

In [10]:
def fahr_to_celsius(temp_fahrenheit):
    """Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8
    
    return converted_temp

Now we have the function defined and stored in memory. At this point it is good to test the function with some known value.

In [11]:
fahr_to_celsius(32)

0.0

32 Fahrenheits is indeed 0 Celsius, so our function seem to be working correctly.

### Using the function by iterating over rows

Next we will learn how to use our function with data stored in pandas DataFrame. We will first apply the function row-by-row using a `for` loop and then we will learn a more efficient way of applying the function to all rows at once.

Looping over rows in a DataFrame can be done in a couple of different ways. A common approach is to use a `iterrows()` method which loops over the rows as a index-Series pairs. In other words, we can use the `iterrows()` method together with a `for` loop to repeat a process *for each row in a Pandas DataFrame*. Please note that iterating over rows this way is a rather inefficient approach, but it is still useful to understand the logic behind the iteration. Other commonly used and faster way to iterate over rows is to use `itertuples()` method. Next, we will see examples from both of them.

When using the `iterrows()` method it is important to understand that `iterrows()` accesses not only the values of one row, but also the `index` of the row as we mentioned. Let's start with a simple for loop that goes through each row in our DataFrame:

In [12]:
# Iterate over the rows
for idx, row in data.iterrows():
    
    # Print the index value
    print('Index:', idx)
    
    # Print the temperature from the row
    print('Temp F:', row['TEMP_F'], "\n")
    
    break

Index: 0
Temp F: 34.0 



We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`. Notice, that when developing a for loop, you don't always need to go through the entire loop if you just want to test things out. Using the `break` statement in Python terminates a loop whenever it is placed inside a loop. We used it here just to test check out the values on the first row. With a large data, you might not want to print out thousands of values to the screen!

Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values in that column using the `fahr_to_celsius()` function that we defined earlier. For updating the value, we can use `at` which we already used earlier in this chapter. This time, we will use the `itertuples()` method which works in a similar manner, except it only return the row values without the `index`. When using `itertuples()` accessing the row values needs to be done a bit differently, because the row is not a Series, but a `named tuple` (hence the name). A tuple is like a list (but immutable, i.e. you cannot change it) and "named tuple" is a special kind of tuple object that adds the ability to access the values by name instead of position index. Hence, below we will access the `TEMP_F` value by using `row.TEMP_F` (compare to how we accessed the value in the prevous code block). 

In [54]:
# Create an empty column for the output values
data['TEMP_C'] = 0.0

# Iterate over the rows 
for row in data.itertuples():
    
    # Convert the Fahrenheit to Celsius
    # Notice how we access the row value
    celsius = fahr_to_celsius(row.TEMP_F)
    
    # Update the value for 'Celsius' column with the converted value
    # Notice how we can access the Index value
    data.at[row.Index, 'TEMP_C'] = celsius

In [55]:
# Check the result
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,MONTH,YEAR
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN,1.111111,190601010600,1,1906
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN,0.000000,190601011300,1,1906
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN,-1.111111,190601012000,1,1906
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN,0.555556,190601020600,1,1906
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN,1.666667,190601021300,1,1906


In [56]:
# How does our row look like?
row

Pandas(Index=198333, STATION_NUMBER=29820, TIME=201910012300, DIR=287.0, SPEED=33.0, GUST=41.0, TEMP_F=46.0, MAX=nan, MIN=nan, TEMP_C=0.0, TIME_STR='201910012300', MONTH=0, YEAR=2019)

Okay, now we have iterated over our data and updated the temperatures in Celsius to `TEMP_C` column by using our `fahr_to_celsius()` function. The values look correct as 32 Fahrenheits indeed is 0 Celsius degrees, as can be seen on the second row. We also have here the last row of our DataFrame which is a named tuple. As you can see, it is a bit like a weird looking dictionary with values assigned to the names of our columns. Basically, it is an object with attributes that we can access in a similar manner as we have used to access some of the pandas DataFrame attributes, such as `data.shape`. 

A couple of notes about our appoaches. We used `itertuples()` method for looping over the values because it is significantly faster compared to `iterrows()` (can be ~100x faster). We used `.at` to assign the value to the DataFrame because it is designed to access single values more efficiently compared to `.loc`, which can access also groups of rows and columns. That said, you could have used `data.loc[idx, new_column] = celsius` to achieve the same result (it is just slower). 

### Using functions with `apply`

Although using for loop with `itertuples()` can be fairly efficient, pandas DataFrames and Series have a dedicated method called `apply()` for applying functions on columns (or rows). `apply()` is typically faster than `itertuples()`, especially if you have large number of rows, such as in our case. When using `apply()`, we pass the function that we want to use as an argument. Let's start by applying the function to the `TEMP_F` column that contains the temperature values in Fahrenheit:

In [15]:
data['TEMP_F'].apply(fahr_to_celsius)

0         1.111111
1         0.000000
2        -1.111111
3         0.555556
4         1.666667
            ...   
198329    8.333333
198330    8.333333
198331    8.333333
198332    8.333333
198333    7.777778
Name: TEMP_F, Length: 198334, dtype: float64

The results look logical. Notice how we passed the `fahr_to_celsius()` function without using the parentheses `()` after the name of the function. When using `apply`, you should always leave out the parentheses from the function that you use. Meaning that you should use `apply(fahr_to_celsius)` instead of `apply(fahr_to_celsius())`. Why? Because the `apply()` method will execute and use the function itself in the background when it operates with the data. If we would pass our function with the parentheses, the `fahr_to_celsius()` function would actually be executed once before the loop with `apply()` starts (hence becoming unusable), and that is not what we want.   

Our command above only returns the Series of temperatures to the screen, but naturally we can also store them permanently into a new column (overwriting the old values). 

In [16]:
data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

A nice thing with `apply()` is that we can also apply the function on several columns at once. Below, we also sort the values in descending order based on values in `MIN` column to see that applying our function really works.  

In [17]:
multiple_columns_at_once = data[['TEMP_F', 'MIN', 'MAX']].apply(fahr_to_celsius)
multiple_columns_at_once.sort_values(by="MIN", ascending=False).head()

,TEMP_F,MIN,MAX
196775,25.000000,25.000000,29.444444
154531,25.555556,24.444444,27.777778
188167,25.000000,24.444444,27.777778
188407,23.888889,23.888889,27.777778
188143,24.444444,23.888889,28.888889


You can also directly store the outputs to new columns `'TEMP_C'`, `'MIN_C'`, `'MAX_C'`.

In [18]:
data[['TEMP_C', 'MIN_C', 'MAX_C']]  = data[['TEMP_F', 'MIN', 'MAX']].apply(fahr_to_celsius)
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,MIN_C,MAX_C
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN,1.111111,NaN,NaN
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN,0.000000,NaN,NaN
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN,-1.111111,NaN,NaN
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN,0.555556,NaN,NaN
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN,1.666667,NaN,NaN


We showed you a few different ways to iterate over rows in pandas and apply functions. The most important thing is that you understand the logic of how loops work and how you can use your own functions to modify the values in a pandas DataFrame. Whenever you need to loop over your data, we recommend using `.apply()` as it is typically the most efficient one in terms of execution time. However, remember that in most cases you do not actually need to use loops, but you can do calculations in a "vectorized manner" (which is the fastest way) as we learned previously when doing basic calculations in pandas. 

## String slicing

We will eventually want to group our data based on month in order to see if the January temperatures in 2020 were higher than on average (which is the goal in our analysis as you might recall). Currently, the date and time information is stored in the column `TIME` (which was originally titled `YR--MODAHRMN`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type.

In [57]:
data['TIME'].head()

0    190601010600
1    190601011300
2    190601012000
3    190601020600
4    190601021300
Name: TIME, dtype: int64

In [58]:
data['TIME'].tail()

198329    201910011900
198330    201910012000
198331    201910012100
198332    201910012200
198333    201910012300
Name: TIME, dtype: int64

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `190601010600`, i.e. from 1st of January 1906 (way back!), and the timestamp for the latest observation is `201910012350`. (**TODO: UPDATE THESE WITH NEW DATA**). As we can see, the data type (`dtype`) of our column seems to be `int64`, i.e. the information is stored as integer values. 

We want to aggregate this data on a monthly level. In order to do so, we need to "label" each row of data based on the month when the record was observed. Hence, we need to somehow separate information about the year and month for each row. In practice, we can create a new column (or an index) containing information about the month (including the year, but excluding days, hours and minutes). There are different ways of achieving this, but here we will take advantage of `string slicing` which means that we convert the date and time information into character strings and "cut" the needed information from the string objects. The other option would be to convert the timestamp values into something called `datetime` objects, but we will learn about those a bit later. 

Before further processing, we first want to convert the `TIME` column as character strings for convenience, stored into a new column `TIME_STR`.

In [60]:
data['TIME_STR'] = data['TIME'].astype(str)

If we look at the latest time stamp in the data (`201910012350`) (**UPDATE!**), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and the following two characters represent month. Because we are interested in understanding monthly averages for different years, we want to slice the year and month values from the timestamp (i.e. the first 6 characters). 

In [61]:
date = "201910012350"
date[0:6]

'201910'

Based on this information, we can slice the correct range of characters from the `TIME_STR` column using a specific pandas function designed for Series, called `.str.slice()`. As parameters the function has `start` and `stop` which you can use to specify the positions where the slicing should start and end.

In [62]:
data['YEAR_MONTH'] = data['TIME_STR'].str.slice(start=0, stop=6)
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,MONTH,YEAR,YEAR_MONTH
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN,1.111111,190601010600,1,1906,190601
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN,0.000000,190601011300,1,1906,190601
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN,-1.111111,190601012000,1,1906,190601
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN,0.555556,190601020600,1,1906,190601
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN,1.666667,190601021300,1,1906,190601


Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day.

#### Check your understanding

Create a new column `'MONTH'` with information about the month without the year.

In [26]:
# Extract information about month from the TIME_STR column into a new column 'MONTH':


## Grouping and aggregating data

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

We will use the groupby method to calculate the average temperatures for each month trough these main steps:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) 
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [28]:
print("number of rows:", len(data))

number of rows: 198334


In [29]:
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,MIN_C,MAX_C,TIME_STR,YEAR_MONTH,MONTH
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN,1.111111,NaN,NaN,190601010600,190601,01
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN,0.000000,NaN,NaN,190601011300,190601,01
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN,-1.111111,NaN,NaN,190601012000,190601,01
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN,0.555556,NaN,NaN,190601020600,190601,01
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN,1.666667,NaN,NaN,190601021300,190601,01


We have quite a few rows of weather data, and several observations per day. Our goal is to create an aggreated data frame that would have only one row per month.

Let's **group** our data based on unique year and month combinations

In [30]:
grouped = data.groupby('YEAR_MONTH')

````{note}
It would be also possible to create combinations of years and months on-the-fly when grouping the data:
    
```
# Group the data 
grouped = data.groupby(['YEAR', 'MONTH'])
```
````


Let's explore the new variable `grouped`:

```python
type(grouped)
```

In [31]:
len(grouped)

826

We have a new object with type `DataFrameGroupBy` with 82 groups. In order to understand what just happened, let's also check the number of unique year and month combinations in our data:

In [32]:
data['YEAR_MONTH'].nunique()

826

Length of the grouped object should be the same as the number of unique values in the column we used for grouping. For each unique value, there is a group of data.

Let's explore our grouped data further. 

Check the "names" of each group

In [33]:
# Next line will print out all 82 group "keys"
#grouped.groups.keys()

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` method:

In [34]:
# Specify a month (as character string)
month = "190601"

# Select the group
group1 = grouped.get_group(month)

In [35]:
# Let's see what we have
group1

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,MIN_C,MAX_C,TIME_STR,YEAR_MONTH,MONTH
0,29820,190601010600,180.0,2.0,NaN,34.0,NaN,NaN,1.111111,NaN,NaN,190601010600,190601,01
1,29820,190601011300,20.0,6.0,NaN,32.0,NaN,NaN,0.000000,NaN,NaN,190601011300,190601,01
2,29820,190601012000,320.0,6.0,NaN,30.0,NaN,NaN,-1.111111,NaN,NaN,190601012000,190601,01
3,29820,190601020600,320.0,10.0,NaN,33.0,NaN,NaN,0.555556,NaN,NaN,190601020600,190601,01
4,29820,190601021300,230.0,15.0,NaN,35.0,NaN,NaN,1.666667,NaN,NaN,190601021300,190601,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,29820,190601301300,360.0,6.0,NaN,28.0,NaN,NaN,-2.222222,NaN,NaN,190601301300,190601,01
89,29820,190601302000,360.0,2.0,NaN,21.0,NaN,NaN,-6.111111,NaN,NaN,190601302000,190601,01
90,29820,190601310600,340.0,6.0,NaN,18.0,NaN,NaN,-7.777778,NaN,NaN,190601310600,190601,01
91,29820,190601311300,340.0,6.0,NaN,30.0,NaN,NaN,-1.111111,NaN,NaN,190601311300,190601,01


Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

In [36]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this specific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` function that we already used during Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [37]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       230.322581
SPEED      13.473118
GUST             NaN
TEMP_F     31.032258
TEMP_C     -0.537634
dtype: float64


Here we saw how you can access data from a single group. For getting information about all groups (all months) we can use a `for` loop or methods available in the grouped object.

**For loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy`  object it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).

In [38]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Key:
 190601

First rows of data in this group:
    STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29820  190601010600  180.0    2.0   NaN    34.0  NaN  NaN   
1           29820  190601011300   20.0    6.0   NaN    32.0  NaN  NaN   
2           29820  190601012000  320.0    6.0   NaN    30.0  NaN  NaN   
3           29820  190601020600  320.0   10.0   NaN    33.0  NaN  NaN   
4           29820  190601021300  230.0   15.0   NaN    35.0  NaN  NaN   

     TEMP_C  MIN_C  MAX_C      TIME_STR YEAR_MONTH MONTH  
0  1.111111    NaN    NaN  190601010600     190601    01  
1  0.000000    NaN    NaN  190601011300     190601    01  
2 -1.111111    NaN    NaN  190601012000     190601    01  
3  0.555556    NaN    NaN  190601020600     190601    01  
4  1.666667    NaN    NaN  190601021300     190601    01  


Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeat slightly the earlier steps so that you can see and better understand what is happening.

In [39]:
# Create an empty DataFrame for the aggregated values
monthly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C']

# Iterate over the groups
for key, group in grouped:
    
   # Calculate mean
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['YEAR_MONTH'] = key

   # Append the aggregated values into the DataFrame
   monthly_data = monthly_data.append(mean_values, ignore_index=True)

- Let's see what we have now:

In [40]:
print(monthly_data)

            DIR       GUST      SPEED     TEMP_C     TEMP_F YEAR_MONTH
0    230.322581        NaN  13.473118  -0.537634  31.032258     190601
1    186.582278        NaN  12.369048  -1.044974  30.119048     190602
2    244.835165        NaN  10.645161  -2.485066  27.526882     190603
3    222.954545        NaN   6.077778   2.740741  36.933333     190604
4    167.500000        NaN   5.847826  10.722820  51.301075     190605
..          ...        ...        ...        ...        ...        ...
821  209.000000  22.211720  17.232172  14.990715  58.983287     201906
822  220.430642  19.846535  15.227743  17.288769  63.119785     201907
823  211.044474  20.131746  15.683288  17.747080  63.944744     201908
824  197.193007  23.123779  17.569231  13.132371  55.638268     201909
825  278.583333  39.791667  33.250000   8.750000  47.750000     201910

[826 rows x 6 columns]


Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [41]:
grouped.mean()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,MIN_C,MAX_C
YEAR_MONTH,,,,,,,,,,,
190601,29820.0,1.906012e+11,230.322581,13.473118,NaN,31.032258,NaN,NaN,-0.537634,NaN,NaN
190602,29820.0,1.906021e+11,186.582278,12.369048,NaN,30.119048,NaN,NaN,-1.044974,NaN,NaN
190603,29820.0,1.906032e+11,244.835165,10.645161,NaN,27.526882,NaN,NaN,-2.485066,NaN,NaN
190604,29820.0,1.906042e+11,222.954545,6.077778,NaN,36.933333,NaN,NaN,2.740741,NaN,NaN
190605,29820.0,1.906052e+11,167.500000,5.847826,NaN,51.301075,NaN,NaN,10.722820,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
201906,29820.0,2.019062e+11,209.000000,17.232172,22.211720,58.983287,62.017544,55.929825,14.990715,13.294347,16.676413
201907,29820.0,2.019072e+11,220.430642,15.227743,19.846535,63.119785,67.216667,59.550000,17.288769,15.305556,19.564815
201908,29820.0,2.019082e+11,211.044474,15.683288,20.131746,63.944744,66.822581,60.919355,17.747080,16.066308,19.345878


### Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all Aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

In [42]:
aprils = data[data['MONTH']=="04"]

- take a subset of columns that might contain interesting information:

In [43]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR_MONTH']]

- group by year and month:

In [44]:
grouped = aprils.groupby(by='YEAR_MONTH')

- calculate mean for each group:

In [45]:
monthly_mean = grouped.mean()

In [46]:
monthly_mean.head()

,STATION_NUMBER,TEMP_F,TEMP_C
YEAR_MONTH,,,
190604,29820,36.933333,2.740741
190704,29820,34.433333,1.351852
190804,29820,36.566667,2.537037
190904,29820,32.611111,0.339506
191004,29820,39.595506,4.219725


- check the highest temperature values (sort the data frame in a descending order):

In [47]:
monthly_mean.sort_values(by='TEMP_C', ascending=False).head(10)

,STATION_NUMBER,TEMP_F,TEMP_C
YEAR_MONTH,,,
201904,29820,41.182197,5.101221
199004,29820,41.144681,5.080378
201404,29820,40.497908,4.721060
200804,29820,39.941423,4.411901
191304,29820,39.622222,4.234568
191004,29820,39.595506,4.219725
200704,29820,39.481013,4.156118
200004,29820,39.210300,4.005722
201604,29820,39.077994,3.932219


How did April 2019 rank at the Tampere Pirkkala observation station? 

## Automating the analysis

Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info - all in one long code cell:

In [48]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, 
                   usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], 
                   na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=['YEAR', 'MONTH'])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by='TEMP_C', ascending=False).head(5))
print("\n")

STATION NUMBER: 29820
NUMBER OF OBSERVATIONS: 198334
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29820  41.182197  5.101221
1990 4               29820  41.144681  5.080378
2014 4               29820  40.497908  4.721060
2008 4               29820  39.941423  4.411901
1913 4               29820  39.622222  4.234568




We will use the `glob()` function from the module `glob` to list our input files. 

In [49]:
import glob

In [50]:
file_list = glob.glob(r'data/0*txt')

```{note}
Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
```

In [51]:
print("Number of files in the list", len(file_list))
print(file_list)

Number of files in the list 1
['data/029820.txt']


Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [52]:
for fp in file_list:
    print(fp)

data/029820.txt


In [53]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=['YEAR', 'MONTH'])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by='TEMP_C', ascending=False).head(5))
    print("\n")

STATION NUMBER: 29820
NUMBER OF OBSERVATIONS: 198334
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29820  41.182197  5.101221
1990 4               29820  41.144681  5.080378
2014 4               29820  40.497908  4.721060
2008 4               29820  39.941423  4.411901
1913 4               29820  39.622222  4.234568




How about now, how did April 2019 rank across different stations?

## Footnotes

[^noaanews]: <https://www.noaa.gov/news/january-2020-was-earth-s-hottest-january-on-record>